In [1]:
!pip3 install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [5]:
tokens = tokenizer.encode('Bad', return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[ 2.9174,  1.9091,  0.5061, -1.9228, -2.6553]],
       grad_fn=<AddmmBackward>)

In [8]:
int(torch.argmax(result.logits))+1

1

In [9]:
import numpy as np
import pandas as pd

In [10]:
data = pd.read_csv('All.csv')
data

,Person,Section,UX Property,Question/Answer
0,Interviewer,Intro,Purpose,Why do you use ARMS?
1,PPT01,Intro,Purpose,"To pay monthly utility, both electricity and w..."
2,Interviewer,Intro,Frequency,How often do you use ARMS?
3,PPT01,Intro,Frequency,One-time thing on a monthly basis
4,Interviewer,Intro,Overall Experience,"In general, as you happy with ARMS' service?"
...,...,...,...,...
100,PPT02,Conclusion,Spillover Effects,"No. If anything, paying bills was smoother, I ..."
101,Interviewer,Conclusion,Preferred Website Feature,Can you think about one thing you like the mos...
102,PPT02,Conclusion,Preferred Website Feature,Yeah definitely I can get estimates of my bill...
103,Interviewer,Conclusion,Recommendation,Would you recommend ARMS to a family member or...


In [11]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [12]:
sentiment_score(data['Question/Answer'].iloc[100])

2

In [13]:
data['sentiment'] = data['Question/Answer'].apply(lambda x: sentiment_score(x[:512]))

In [14]:
data

,Person,Section,UX Property,Question/Answer,sentiment
0,Interviewer,Intro,Purpose,Why do you use ARMS?,1
1,PPT01,Intro,Purpose,"To pay monthly utility, both electricity and w...",5
2,Interviewer,Intro,Frequency,How often do you use ARMS?,2
3,PPT01,Intro,Frequency,One-time thing on a monthly basis,4
4,Interviewer,Intro,Overall Experience,"In general, as you happy with ARMS' service?",3
...,...,...,...,...,...
100,PPT02,Conclusion,Spillover Effects,"No. If anything, paying bills was smoother, I ...",2
101,Interviewer,Conclusion,Preferred Website Feature,Can you think about one thing you like the mos...,3
102,PPT02,Conclusion,Preferred Website Feature,Yeah definitely I can get estimates of my bill...,5
103,Interviewer,Conclusion,Recommendation,Would you recommend ARMS to a family member or...,3
